In [1]:
import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from discussion_agents.cog.agent.reflexion import ReflexionCoTAgent
from langchain_community.chat_models.openai import ChatOpenAI

In [3]:
agent = ReflexionCoTAgent(
    self_reflect_llm=ChatOpenAI(openai_api_key=openai_api_key),
    action_llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [4]:
q = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'


In [5]:
out = agent.generate(q, key="Badr Hari")

In [10]:
print("\n".join(out[0][-1]))

Thought: Let's think step by step. The description fits the kickboxer Badr Hari, who was once considered the best in the world but has faced controversies for his behavior both in and out of the ring.Action: Finish[Badr Hari]
Action: Finish[Badr Hari]
Observation: Answer is CORRECT
